In [1]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
def build_network(width, height, depth, classes):
    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32,
               kernel_size=(3,3),
               padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3,3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(rate=0.25)(x)\
    
    x = Conv2D(filters=64,
               kernel_size=(3,3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3,3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=classes)(x)
    output = Softmax()(x)

    return Model(input_layer, output)

In [4]:
def plot_model_history(model_history, metric, plot_name):
    plt.style.use('seaborn-darkgrid')
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)
    plt.title(f'{metric.upper()}')
    plt.ylim([0, 1])
    plt.savefig(f'{plot_name}.png')
    plt.close()

In [5]:
def load_image_and_label(image_path, target_size=(64,64)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CLASSES)
    label = tf.dtypes.cast(label, tf.float32)
    
    return image, label

In [6]:
def augment(image, label):
    image = tf.image.resize_with_crop_or_pad(image, 74, 74)
    image = tf.image.random_crop(image, size=(64,64,3))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)

    return image, label

In [7]:
def prepare_dataset(data_pattern):
    return (tf.data.Dataset
            .from_tensor_slices(data_pattern)
            .map(load_image_and_label,
                 num_parallel_calls=AUTOTUNE))

In [8]:
SEED = 999
np.random.seed(SEED)

In [9]:
base_path = ('C:/Users/hp/Documents/DATA/caltech-101/101_ObjectCategories/*/*.jpg')
image_pattern = str(base_path)
image_paths = [*glob.glob(image_pattern)]
image_paths = [p for p in image_paths if
               p.split(os.path.sep)[-2] !=
               'BACKGROUND_Google']

In [10]:
CLASSES = np.unique([p.split(os.path.sep)[-2]
                     for p in image_paths])

In [11]:
train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=SEED)

In [12]:
BATCH_SIZE = 64
BUFFER_SIZE = 1024
train_dataset = (prepare_dataset(train_paths)
                 .batch(BATCH_SIZE)
                 .shuffle(buffer_size=BUFFER_SIZE)
                 .prefetch(buffer_size=BUFFER_SIZE))
test_dataset = (prepare_dataset(test_paths)
                .batch(BATCH_SIZE)
                .prefetch(buffer_size=BUFFER_SIZE))


In [13]:
EPOCHS = 10
model = build_network(64, 64, 3, len(CLASSES))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=EPOCHS)
result = model.evaluate(test_dataset)
print(f"test accuracy: {result[1]}")
plot_model_history(history, 'accuracy', 'normal')

Epoch 1/10
109/109 [==============================] - 122s 1s/step - loss: 3.0434 - accuracy: 0.3910 - val_loss: 4.8967 - val_accuracy: 0.1152
Epoch 2/10
109/109 [==============================] - 116s 1s/step - loss: 1.6238 - accuracy: 0.6160 - val_loss: 4.6496 - val_accuracy: 0.1267
Epoch 3/10
109/109 [==============================] - 114s 1s/step - loss: 0.9253 - accuracy: 0.7757 - val_loss: 3.2794 - val_accuracy: 0.3312
Epoch 4/10
109/109 [==============================] - 114s 1s/step - loss: 0.4584 - accuracy: 0.8957 - val_loss: 2.0080 - val_accuracy: 0.5536
Epoch 5/10
109/109 [==============================] - 116s 1s/step - loss: 0.2305 - accuracy: 0.9572 - val_loss: 1.5535 - val_accuracy: 0.6244
Epoch 6/10
109/109 [==============================] - 117s 1s/step - loss: 0.1170 - accuracy: 0.9806 - val_loss: 1.4156 - val_accuracy: 0.6642
Epoch 7/10
109/109 [==============================] - 118s 1s/step - loss: 0.0727 - accuracy: 0.9908 - val_loss: 1.4717 - val_accuracy: 0.6607

C:\Users\hp\AppData\Local\Temp\ipykernel_3488\3768594648.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [14]:
train_dataset = (prepare_dataset(train_paths)
                 .map(augment,
                      num_parallel_calls=AUTOTUNE)
                      .batch(BATCH_SIZE)
                      .shuffle(buffer_size=BUFFER_SIZE)
                      .prefetch(buffer_size=BUFFER_SIZE))
test_dataset = (prepare_dataset(test_paths)
                .batch(BATCH_SIZE)
                .prefetch(buffer_size=BUFFER_SIZE))

In [15]:
model = build_network(64, 64 , 3, len(CLASSES))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=EPOCHS)
result = model.evaluate(test_dataset)
print(f"test accuracy: {result[1]}")
plot_model_history(history, 'accuracy', 'augmented')

Epoch 1/10
109/109 [==============================] - 132s 1s/step - loss: 3.6516 - accuracy: 0.2884 - val_loss: 5.7860 - val_accuracy: 0.0063
Epoch 2/10
109/109 [==============================] - 116s 1s/step - loss: 2.6436 - accuracy: 0.4152 - val_loss: 7.3837 - val_accuracy: 0.0628
Epoch 3/10
109/109 [==============================] - 119s 1s/step - loss: 2.2108 - accuracy: 0.4911 - val_loss: 4.0718 - val_accuracy: 0.1694
Epoch 4/10
109/109 [==============================] - 127s 1s/step - loss: 1.8835 - accuracy: 0.5534 - val_loss: 2.8885 - val_accuracy: 0.4217
Epoch 5/10
109/109 [==============================] - 118s 1s/step - loss: 1.7541 - accuracy: 0.5746 - val_loss: 1.8708 - val_accuracy: 0.5490
Epoch 6/10
109/109 [==============================] - 120s 1s/step - loss: 1.6914 - accuracy: 0.5905 - val_loss: 2.2412 - val_accuracy: 0.5426
Epoch 7/10
109/109 [==============================] - 122s 1s/step - loss: 1.5236 - accuracy: 0.6201 - val_loss: 1.5846 - val_accuracy: 0.6037

C:\Users\hp\AppData\Local\Temp\ipykernel_3488\3768594648.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')
